# NFL Big Data Bowl 2024

## Attempting to predict direction and pass/run of the play based on pre-snap information.

## Importing dataview libraries

In [180]:
import re
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [164]:
DATA_ROOT = '../data/'

In [165]:
plays = pd.read_csv(DATA_ROOT + 'plays.csv')

## Data cleanup and splitting

In [188]:
def playDescriptionToDirection(play_description):
    play = re.search('(pass (?:short|deep) (?:left|middle|right))|((?:left|right) (?:guard|tackle|end))|(up the middle)', play_description)
    return play.group(0) if play else 'FUMBLED BALL'

In [391]:
plays['playDirection'] = plays['playDescription'].map(playDescriptionToDirection)
plays['overTenYards'] = plays['yardsToGo'] > 10

numeric_feature_names = ['yardsToGo', 'defendersInTheBox', 'absoluteYardlineNumber']
categoric_feature_names = ['offenseFormation', 'down']
binary_feature_names = ['overTenYards']
target_name = 'playDirection'

df = plays[numeric_feature_names + categoric_feature_names + binary_feature_names + [target_name]].copy()
df = df.dropna()

# Remove fumbled ball as it only occurs 6 times.
fumble_mask = df['playDirection'] == 'FUMBLED BALL'
df = df[~fumble_mask]

first_downs = df[(df['down'] == 1)]
df = df[~(df['down'] == 1)]

# Undersample to balance the data.
# df_group = df.groupby(target_name)
# df_balanced = df_group.apply(lambda x: x.sample(df_group.size().min()).reset_index(drop=True))
# df = df_balanced

# One-hot encode categoric features.
df = pd.get_dummies(df, columns=categoric_feature_names)

# Shuffle the data.
df = df.iloc[np.random.permutation(len(df))].reset_index(drop=True)

target = df.pop(target_name)
target_raw = target.copy()
target = pd.get_dummies(target)

input_dim = len(df.columns)
output_dim = len(target.columns)

In [367]:
# Split into train, validation, and test sets.
x, x_val = df[:int(len(df) * 0.8)], df[int(len(df) * 0.8):]
y, y_val = target[:int(len(target) * 0.8)], target[int(len(target) * 0.8):]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
x_val = np.asarray(x_val).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_val = np.asarray(y_val).astype('float32')

# TF Neural Net Approach

In [406]:
epochs = 5
learning_rate = 0.01
batch_size = 64
layer_count = 5

def make_sequential_net(layers, epochs, batch_size, learning_rate):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Input(shape=(input_dim,)))
  for i in range(layer_count):
      # Use input_size * (2/3) + output_size for hidden layer size.
      model.add(tf.keras.layers.Dense(input_dim*(2/3)+output_dim, activation='relu'))
  model.add(tf.keras.layers.Dense(output_dim, activation='softmax'))

  opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate, decay=1e-5)

  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
  loss, acc = model.evaluate(x_val, y_val)
  return model, acc

model, acc = make_sequential_net(layer_count, epochs, batch_size, learning_rate)

Epoch 1/5
10/10 [==============================] - 3s 163ms/step - loss: 2.4003 - accuracy: 0.0582
Epoch 2/5
10/10 [==============================] - 0s 52ms/step - loss: 1.6119 - accuracy: 0.0719
Epoch 3/5
10/10 [==============================] - 0s 25ms/step - loss: 1.0585 - accuracy: 0.0839
Epoch 4/5
10/10 [==============================] - 0s 42ms/step - loss: 0.9543 - accuracy: 0.0685
Epoch 5/5
7/7 [==============================] - 1s 87ms/step - loss: 0.8370 - accuracy: 0.0776


## SKLearn Decision Tree Classifier

In [407]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
depth = 10
leaf = 20
clf = DecisionTreeClassifier(max_depth=depth, max_leaf_nodes=leaf)
score = np.mean(cross_val_score(clf, df, target_raw, cv=10))
print(score)

0.22092103797750307


## SKLearn Random Forest Approach

In [133]:
from sklearn.ensemble import RandomForestClassifier

In [409]:
clf = RandomForestClassifier(max_depth=70, max_leaf_nodes=20)
score = np.mean(cross_val_score(clf, df, target_raw, cv=10))
print(score)

0.22853510041949737
